In [1]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100
%matplotlib inline

In [2]:
image1 = cv2.imread("climbing_1_small.jpg")
image2 = cv2.imread("climbing_2_small.jpg")
gray1 = cv2.imread("climbing_1_small.jpg", cv2.IMREAD_GRAYSCALE)
gray2 = cv2.imread("climbing_2_small.jpg", cv2.IMREAD_GRAYSCALE)
sift = cv2.SIFT()

In [3]:
# find the keypoints and descriptors with SIFT 
keypoint1, descriptors1 = sift.detectAndCompute(gray1,None) 
keypoint2, descriptors2 = sift.detectAndCompute(gray2,None)

In [4]:
# FLANN parameters 
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5) 
search_params = dict(checks=50)  # or pass empty dictionary 
flann = cv2.FlannBasedMatcher(index_params,search_params) 
matches = flann.knnMatch(descriptors1,descriptors2,k=2) 

In [5]:
# Apply ratio test 
good = [] 
for m,n in matches:
    if m.distance < 0.45*n.distance:
        good.append(m)
print 'good',len(good) 

good 67


In [6]:
h1, w1 = image1.shape[:2]
h2, w2 = image2.shape[:2]

h1, w1 = image1.shape[:2] 
h2, w2 = image2.shape[:2]


view = sp.zeros((max(h1, h2), w1 + w2, 3), np.uint8)

view[:h1, :w1, :] = image1
view[:h2, w1:, :] = image2

In [7]:
for m in good:
    # draw the keypoints
    color = tuple([np.random.randint(0, 255) for _ in xrange(3)])
    cv2.line(
        view,
        (int(keypoint1[m.queryIdx].pt[0]), int(keypoint1[m.queryIdx].pt[1])),
        (int(keypoint2[m.trainIdx].pt[0] + w1), int(keypoint2[m.trainIdx].pt[1])),
        color,3
    ) 

In [8]:
while True:
    cv2.imshow('image', view)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()